In [1]:
from networkx import maximum_flow
from networkx import gnm_random_graph
from networkx import draw 
from networkx import adjacency_matrix
from networkx import all_simple_paths
from networkx import shortest_path
from networkx import set_edge_attributes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Graph

In [150]:
# Density is the proportion of edges in a full connected graph (dense graph has n(n-1) edges where n is n_nodes)
# maximum_flow from networkx has a linear trend wrt density.
# Column generation has an exponential trend wrt density (keep a density < 0.3 !!!)

min_weight = 5
max_weight = 15
n_nodes = 10 ** 3
n_edges = 8 * 10 ** 5
density = n_edges / (n_nodes * (n_nodes - 1))
print ("density = ", density)
source, destination = 0, 1 # don't need to randomize (graph is randomized)

density =  0.8008008008008008


## Creates a random valuated graph

In [151]:
def modify_weights_graph (Graph, ctr_lists, new_weights) :
    """ utility function for setting weights (dual problem of shortest path) """
    set_edge_attributes(Graph, 0, name="weight")

    set_edge_attributes(
        Graph,
        values = dict({edge : dual for edge, dual in zip (ctr_lists, new_weights)}),
        name = 'weight'
    )

In [152]:
Graph = gnm_random_graph(n_nodes, n_edges, directed=True)
Graph_dual = deepcopy(Graph)

for (u, v) in Graph.edges():
    Graph.edges[u,v]['weight'] = np.random.uniform(min_weight, max_weight)

In [153]:
Adj_matrix = adjacency_matrix(Graph)
Adj_matrix.shape

C:\Users\val\AppData\Local\Temp\ipykernel_9616\2680887831.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj_matrix = adjacency_matrix(Graph)


(1000, 1000)

In [154]:
flow, flow_dict = maximum_flow(Graph, _s = source, _t = destination, capacity='weight')
# total flow, flow first node
flow, flow_dict[0]

(8054.563833331987,
 {919: 5.235628844991234,
  914: 8.779970065077945,
  554: 6.822377122592699,
  368: 8.067977532199652,
  483: 12.211394894683256,
  282: 11.89201246845347,
  182: 9.837900293481853,
  900: 6.734616941554111,
  204: 6.045792326617087,
  492: 11.940796897599276,
  91: 5.480095963099565,
  111: 6.996110962584185,
  638: 5.832986773095366,
  584: 10.574755852894647,
  928: 5.612692453571656,
  808: 9.03577822651199,
  723: 9.493433668428112,
  359: 7.695762024687051,
  694: 10.640275299048689,
  308: 8.885763399669035,
  968: 9.12997055950454,
  23: 9.29372613655046,
  286: 13.548469197538843,
  293: 11.295510217350758,
  338: 5.619490540771462,
  728: 10.597599429632798,
  619: 9.598332787972248,
  328: 8.774507749460462,
  532: 5.837007562648187,
  509: 10.368280366447127,
  431: 10.102322990895795,
  500: 14.795230505676804,
  426: 11.066618884733181,
  334: 11.944489379352294,
  138: 8.370224230504391,
  165: 12.37444878449437,
  13: 7.769706721786507,
  417: 13.51

Maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [117]:
import pulp as pl

In [118]:
solver = pl.getSolver('CPLEX_CMD', timeLimit=10)
solver.path

'cplex.exe'

In [119]:
def get_n_paths (Graph, source, destination, max_iter = 16):
    """ Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination """
    list_ = []
    for (index, path) in enumerate(all_simple_paths(Graph, source, destination, cutoff=10)):
        list_.append(path)
        if (index >= max_iter - 1):
            return (list_)
    return (list_)

In [120]:
# affichage de n chemins 
len(get_n_paths(Graph, source, destination, max_iter=50))

50

## Master

In [121]:
# Etape 0 INIT THE MASTER PROBLEM

lpProb = pl.LpProblem("path_generation", pl.LpMaximize)
lpProb += 0 # initiate the objective function to 0 (necessary line of code)

In [122]:
# useful to know the final paths at the end and calculate the shortest path for dual problem
path2idx = {}
ctr_lists = []
path_count = 0

In [123]:
def add_paths (lpProb, paths, path_count, path2idx):
    """ Add paths in lpPoblem (new variables Xp and new constraints or terms in constraints Ce) """

    # pour chaque path ajouter une variable et ses coef. dans les contraintes
    for path in paths :

        # add a var path
        prov = pl.LpVariable("X"+str(path_count), 0, max_weight, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)
        
        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path)-1) :
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1) # add term in constraint
            else:
                lpProb.constraints[index_ctr] = prov <= Graph.edges[(path[index], path[index + 1])]
                ctr_lists.append((path[index], path[index + 1]))
        
        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

In [124]:
path_count, path2idx = add_paths (lpProb, get_n_paths (Graph, source, destination, max_iter = 1), path_count, path2idx)

In [125]:
# solver = pl.CPLEX_PY()
result = lpProb.solve()
# result = lpProb.solve(pl.GUROBI_CMD(options=[("MIPgap", 0.9)]))

## Dual variables

In [126]:
def get_duals(lpPb):
    """ Get the dual variables from a problem Pulp """
    return [c.pi for _, c in list(lpPb.constraints.items())]

## Iterative procedure

In [127]:
go_on = True

while go_on :
    # search of path q (dual problem)
    duals = get_duals(lpProb)
    modify_weights_graph (Graph_dual, ctr_lists, duals)
    path_q = shortest_path(Graph_dual, source, destination, weight="weight")

    # compute reduced cost for dual
    reduced_cost = 1
    for e in range(len(path_q)-1) :
        try :
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError : # <|-- constraint does not exist (so dual == 0)
            pass

    # stop procedure ?
    if reduced_cost > 0 : # add path q in lp Problem
        path_count, path2idx = add_paths(lpProb, [path_q], path_count, path2idx)
        lpProb.solve()
    else :
        go_on = False

In [128]:
flow = 0
for var in lpProb.variables():
    flow += var.value()
print(flow)

4839.437096175298


In [79]:
paths = {path2idx[int(str(var)[1:])] : var.value() for var in lpProb.variables() if var.value() > 0}
paths

{'[0, 2107, 4197, 5055, 5504, 4210, 7559, 1]': 5.7548989,
 '[0, 2107, 8772, 9726, 6270, 656, 8529, 1648, 1]': 6.7632141,
 '[0, 8201, 9066, 2952, 1402, 8048, 1190, 6035, 1]': 5.1518474,
 '[0, 3744, 6023, 3570, 9697, 6926, 1]': 4.4545362,
 '[0, 1917, 8186, 1228, 4366, 4048, 1]': 5.3732747,
 '[0, 1917, 9051, 3701, 5316, 9175, 8161, 1]': 1.9539247,
 '[0, 3744, 6023, 6431, 1189, 9383, 6076, 6035, 1]': 1.2135782,
 '[0, 6910, 5821, 9267, 1385, 4552, 7559, 1]': 0.51943363,
 '[0, 6910, 3762, 3658, 4283, 1787, 7797, 7384, 1]': 5.5532011,
 '[0, 3744, 4231, 7809, 9018, 3772, 1648, 1]': 5.0259489}